# Temas: Presentar Resultados
<br/><br/>

<center>
    
## Taller de Ciencia de Datos
### Omar Piña Ramírez
### Instituto Nacional de Perinatología
### Departamento de Bioinformática y Análisis Estadísticos
### Investigador en Ciencias Médicas
### delozath@gmail.com
</center>

In [ ]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1500px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
.CodeMirror{
    font-size: 15px;
}

.rendered_html table, .rendered_html td, .rendered_html th {
    font-size: 120%;
}
</style>

In [ ]:
import numpy   as np
import pandas  as pd
import seaborn as sns

from   matplotlib import pyplot as plt

import ipywidgets as widgets
from   ipywidgets import interact, interact_manual, FloatSlider, Layout

import chart_studio.plotly as py
import plotly.graph_objs   as go
import plotly.express      as px
from   plotly.offline      import iplot, init_notebook_mode

import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode (connected=True)

## Gráficas

### Datos Infarto

In [ ]:
PATH  = './data/'
file = 'cardiovascular.csv'

data = pd.read_csv(PATH + file)
data = data.drop(columns='ind')
data

In [ ]:
groups = ['famhist','chd']
cols   = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity','alcohol', 'age']

### Gráficas de caja

In [ ]:
df = pd.melt(data,id_vars=groups[0])
df

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,6))
sns.boxplot(x='variable',y='value',hue=groups[0],data=df,ax=ax)
ax.grid(b=True)
ax.set_title('Boxplot',fontsize=15)
plt.show()

In [ ]:
df = pd.melt(data.drop(columns='famhist'),id_vars=groups[1])
fig, ax = plt.subplots(1,1,figsize=(10,6))
sns.boxplot(x='variable',y='value',hue=groups[1],data=df,ax=ax)
ax.grid(b=True)
ax.set_title('Boxplot',fontsize=15)
plt.show()

### Heatmap: outlayers

In [ ]:
from scipy.stats import zscore

df       = data
fig, ax  = plt.subplots(2,2,figsize=(12,15))
for n,(group, vals) in enumerate( df.groupby(['famhist','chd']) ):
    i,j = n%2, n//2
    df_zscore = vals.drop(columns=['famhist','chd']).apply(zscore)
    #cmap='Spectral'
    sns.heatmap(df_zscore,ax=ax[i,j])
    ax[i,j].set_title("Fam-Hist: {}, chf: {}".format(*group))

plt.show()

### Distribución de clases

In [ ]:
from sklearn.datasets import make_blobs
centers = [(-.5, -.5), (.5, .5)]
samples = [50, 100, 500]
var     = [.5, 1, 2]

@interact
def scatter_plot(n_samples=(50,1000,100),v0=var, v1=var):
    X, y = make_blobs(n_samples=n_samples, centers=centers, cluster_std=[v0,v1] )
    fig  = px.scatter(x=X[:,0], y=X[:,1],color=y.astype('int'),width=600, height=600)
    fig.update_traces(marker={'size': 15})
    fig.show()

In [ ]:
centers = [(-.5, -.5), (.5, .5)]
samples = 500
var     = [.5, 1]

X, y = make_blobs(n_samples=500, centers=centers, cluster_std=var )

fig, ax = plt.subplots(1,1,figsize=(8,8))
sns.scatterplot(x=X[:,0], y=X[:,1], hue=y, s=80)
ax.grid(b=True)

In [ ]:
from sklearn.svm             import SVC
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics         import recall_score as sensitivity

k     = 10
sf    = ShuffleSplit(n_splits=k,test_size=.25)
model = SVC(kernel='linear')

sen_perform = []
spe_perform = []
for train,test in sf.split(X,y):
    model.fit(X[train],y[train].ravel())
    L   = model.predict(X[test])
    sen = sensitivity(y[test],L)
    spe = sensitivity(y[test],L,pos_label=0)
    print("Sensitivity: {:.3f}, Specificity: {:3f}".format(sen,spe))
    sen_perform.append(sen)
    spe_perform.append(spe)

sen_perform = np.array(sen_perform)
spe_perform = np.array(spe_perform)
print("\nPerformance")
print("Sensitivity: {:.3f} ± {:.3f}".format(sen_perform.mean(),sen_perform.std()))
print("Specificity: {:.3f} ± {:.3f}".format(spe_perform.mean(),spe_perform.std()))

In [ ]:
N  = 200
xx = np.linspace(-3, 3.75, N)
XX = np.meshgrid(xx,xx)
XX = np.array([XX[0].ravel(),XX[1].ravel()]).T
L  = model.predict(XX)
L  = L.reshape(N,N)

plt.figure(figsize=(7,7))
plt.imshow(L,origin='lower', cmap='inferno',extent=[-3, 3.75, -3, 3.75])
plt.plot(X[y==1,0],X[y==1,1],'o',color='orange',markersize=8)
plt.plot(X[y==0,0],X[y==0,1],'o',color='blue'  ,markersize=8)
plt.show()

In [ ]:
@interact
def rsvm_distr(gamma=[0.001, 0.01, 0.05, 0.1, 1, 3, 4]):
    model = SVC(kernel='rbf',gamma=gamma)
    
    sen_perform = []
    spe_perform = []
    for train,test in sf.split(X,y):
        model.fit(X[train],y[train].ravel())
        L   = model.predict(X[test])
        sen = sensitivity(y[test],L)
        spe = sensitivity(y[test],L,pos_label=0)
        print("Sensitivity: {:.3f}, Specificity: {:3f}".format(sen,spe))
        sen_perform.append(sen)
        spe_perform.append(spe)
    
    sen_perform = np.array(sen_perform)
    spe_perform = np.array(spe_perform)
    print("\nPerformance")
    print("Sensitivity: {:.3f} ± {:.3f}".format(sen_perform.mean(),sen_perform.std()))
    print("Specificity: {:.3f} ± {:.3f}".format(spe_perform.mean(),spe_perform.std()))
    
    L  = model.predict(XX)
    L  = L.reshape(N,N)
    
    plt.figure(figsize=(7,7))
    plt.imshow(L,origin='lower', cmap='inferno',extent=[-3, 3.75, -3, 3.75])
    plt.plot(X[y==1,0],X[y==1,1],'o',color='orange',markersize=8)
    plt.plot(X[y==0,0],X[y==0,1],'o',color='blue'  ,markersize=8)
    plt.show()

In [ ]:
@interact
def rsvm_distr(gamma=[0.001, 0.01, 0.05, 0.1, 1, 3, 4],C=[0.01, 0.1, 1, 5, 10, 20]):
    model = SVC(kernel='rbf',C=C, gamma=gamma)
    
    sen_perform = []
    spe_perform = []
    for train,test in sf.split(X,y):
        model.fit(X[train],y[train].ravel())
        L   = model.predict(X[test])
        sen = sensitivity(y[test],L)
        spe = sensitivity(y[test],L,pos_label=0)
        #print("Sensitivity: {:.3f}, Specificity: {:3f}".format(sen,spe))
        sen_perform.append(sen)
        spe_perform.append(spe)
    
    sen_perform = np.array(sen_perform)
    spe_perform = np.array(spe_perform)
    print("\nPerformance")
    print("Sensitivity: {:.3f} ± {:.3f}".format(sen_perform.mean(),sen_perform.std()))
    print("Specificity: {:.3f} ± {:.3f}".format(spe_perform.mean(),spe_perform.std()))
    
    L  = model.predict(XX)
    L  = L.reshape(N,N)
    
    plt.figure(figsize=(7,7))
    plt.imshow(L,origin='lower', cmap='inferno',extent=[-3, 3.75, -3, 3.75])
    plt.plot(X[y==1,0],X[y==1,1],'o',color='orange',markersize=8)
    plt.plot(X[y==0,0],X[y==0,1],'o',color='blue'  ,markersize=8)
    plt.show()

In [ ]:
@interact
def rsvm_distr(gamma=[0.001, 0.01, 0.05, 0.1, 1, 3, 4],C=[0.01, 0.1, 1, 5, 10, 20]):
    model = SVC(kernel='rbf',C=C, gamma=gamma, probability=True)
    
    model.fit(X[:400],y[:400].ravel())
    L  = model.predict_proba(XX)[:,0]
    L  = L.reshape(N,N)
    
    plt.figure(figsize=(7,7))
    plt.imshow(L,origin='lower', cmap='inferno',extent=[-3, 3.75, -3, 3.75])
    plt.plot(X[y==1,0],X[y==1,1],'o',color='orange',markersize=8)
    plt.plot(X[y==0,0],X[y==0,1],'o',color='blue'  ,markersize=8)
    plt.show()